<a href="https://colab.research.google.com/github/cra2003/clg/blob/main/QA_with_spider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets
import datasets
from datasets import load_dataset
dataset = load_dataset("squad")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [3]:
dataset['train'] = dataset['train'][:10]


In [4]:
import pandas as pd

train_df = pd.DataFrame(dataset['train'])

In [5]:
train_df

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...
5,5733bf84d058e614000b61be,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",When did the Scholastic Magazine of Notre dame...,"{'text': ['September 1876'], 'answer_start': [..."
6,5733bf84d058e614000b61bf,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",How often is Notre Dame's the Juggler published?,"{'text': ['twice'], 'answer_start': [441]}"
7,5733bf84d058e614000b61c0,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",What is the daily student paper at Notre Dame ...,"{'text': ['The Observer'], 'answer_start': [598]}"
8,5733bf84d058e614000b61bd,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",How many student news papers are found at Notr...,"{'text': ['three'], 'answer_start': [126]}"
9,5733bf84d058e614000b61c1,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",In what year did the student paper Common Sens...,"{'text': ['1987'], 'answer_start': [908]}"


MODEL

In [6]:
from transformers import DPRReader, DPRReaderTokenizer

tokenizer = DPRReaderTokenizer.from_pretrained("tau/spider")
model = DPRReader.from_pretrained("tau/spider")

tokenizer_config.json:   0%|          | 0.00/332 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRReaderTokenizer'.


config.json:   0%|          | 0.00/597 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DPRReader were not initialized from the model checkpoint at tau/spider and are newly initialized: ['encoder.bert_model.embeddings.LayerNorm.bias', 'encoder.bert_model.embeddings.LayerNorm.weight', 'encoder.bert_model.embeddings.position_embeddings.weight', 'encoder.bert_model.embeddings.token_type_embeddings.weight', 'encoder.bert_model.embeddings.word_embeddings.weight', 'encoder.bert_model.encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.bert_model.encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.bert_model.encoder.layer.0.attention.output.dense.bias', 'encoder.

In [7]:
import torch

In [8]:
def generate_predictions_and_add_to_dataframe(dataset):
    predicted_answers = []
    for i, row in dataset.iterrows():
        context = row['context']
        question = row['question']
        title = row['title']
        encoded_inputs = tokenizer(
            questions=[question],
            titles=[title],
            texts=[context],
            return_tensors="pt",
        )
        outputs = model(**encoded_inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits


        start_index = torch.argmax(start_logits)
        end_index = torch.argmax(end_logits)
        predicted_answer = context[start_index:end_index+1]


        predicted_answers.append(predicted_answer)


    dataset['predicted_answer'] = predicted_answers
    return dataset


train_df_with_predictions = generate_predictions_and_add_to_dataframe(train_df)


print(train_df_with_predictions.head())

                         id                     title  \
0  5733be284776f41900661182  University_of_Notre_Dame   
1  5733be284776f4190066117f  University_of_Notre_Dame   
2  5733be284776f41900661180  University_of_Notre_Dame   
3  5733be284776f41900661181  University_of_Notre_Dame   
4  5733be284776f4190066117e  University_of_Notre_Dame   

                                             context  \
0  Architecturally, the school has a Catholic cha...   
1  Architecturally, the school has a Catholic cha...   
2  Architecturally, the school has a Catholic cha...   
3  Architecturally, the school has a Catholic cha...   
4  Architecturally, the school has a Catholic cha...   

                                            question  \
0  To whom did the Virgin Mary allegedly appear i...   
1  What is in front of the Notre Dame Main Building?   
2  The Basilica of the Sacred heart at Notre Dame...   
3                  What is the Grotto at Notre Dame?   
4  What sits on top of the Main Building

In [20]:
print(train_df['answers'][0]['text'])

['Saint Bernadette Soubirous']


In [21]:
print(train_df['predicted_answer'][0])

ary. Imm
